In [156]:
### imports and read json file ###

################################################################################

import numpy as np
import random
import json
from pprint import pprint
import time
import sys
from queue import PriorityQueue


#open file
with open('shakespeare-scenes.json') as f:
    data = json.load(f)
    

#pprint(data['corpus'][93])
#'playId': 'merry_wives_of_windsor',
#'sceneId': 'merry_wives_of_windsor:3.4',
#'sceneNum': 93,
#'text': 'scene v a room in the garter inn enter host and simple host what '

In [160]:
### checks/ print ###

################################################################################

#print(invertls['fear'])
#print( metadata['playLen'])

{1: [664], 2: [885], 3: [797, 1663, 2680], 5: [980, 1729], 8: [1516, 1635, 1807, 1897, 2071, 2226, 2233, 2246], 11: [328], 13: [30], 14: [341, 356, 370, 670, 2438, 3223], 15: [760, 777, 825, 855, 1085, 1158, 1619], 17: [668], 21: [1518, 2748], 22: [815], 24: [1246, 1268, 1529], 26: [139], 27: [1038], 28: [1678, 1785], 30: [189, 427], 33: [689, 760, 768], 34: [626, 1629, 1664], 36: [1743], 38: [443, 2596, 2619], 39: [752, 1257], 41: [122], 43: [245, 584, 1450, 1571, 1575, 1593, 1612, 1663], 46: [1264], 47: [580, 582], 48: [600, 602, 609], 49: [383, 510, 1221], 50: [29], 52: [287], 53: [1216, 1313, 1588], 54: [585, 891], 56: [45, 198], 59: [613, 646, 685], 60: [507], 61: [267, 326, 328, 1621, 1635, 1731, 1985, 2855, 2918], 62: [2212], 64: [1020, 1027, 1125], 65: [441, 451, 468], 66: [160], 67: [205, 509], 70: [770], 73: [730], 75: [301], 77: [284, 299, 1428], 82: [211], 86: [624, 994], 88: [825], 92: [352, 704], 95: [209], 100: [1124], 101: [1740], 102: [369, 3065], 104: [1130], 105: [11

In [62]:
### term class ###

################################################################################

class termList(dict):
    def __init__(self, term):
        #remember the meta data
        self.currentDoc = 0 #the doc we are on 
        #self.docinv = {} #{2:[3,5,6], 5:[3,5,6] ...}
        #get incremented when building the list
        self.collection_f = 0  # total number of times a word appears in all of the docs 
        self.name = term
        
    #skip to the nth doc or beyond
    def skipTo(self, goto_doc):
        #if the wanted term is in the dictionary, dic has that key
        if(dic.get(goto_doc)!= None):
            #set current doc to that doc
            self.currentDoc = goto_doc
        #when we cannot find that doc, go to the next doc available
        else:
            #advance 1 till we get to a doc ID that is larger than goto doc
            while(self.currentDoc < goto_doc or dic.get(self.currentDoc)!= None):
                self.currentDoc += 1
            
    
    #add a word to the term list
    #when we find another of the same word for this term
    #given the term name, the position and in which doc we find it
    def addWord(self, word, wordplace, docid):
        #if we already have a doc id for it
        if(self.get(docid)!=None):
            #add to the list
            self[docid].append(wordplace)
            #increment the count
            self.collection_f += 1
            
        else:
            #create a list for that doc
            self[docid] = []
            #add word
            self[docid].append(wordplace)
            #increment the count
            self.collection_f += 1
            
            
        

In [157]:
### global variables ###

################################################################################

#data about the collection as a whole
metadata = {}
metadata['collectionLen'] = 0 # number of doc in the collection
metadata['playLen'] = [] #list of pairs[play length, play name]
metadata['docLen'] = [] #list of pairs[doc length, doc name] 
metadata['docid'] = [] #list of doc names like play:2.2
metadata['terms'] = [] #list of strings of all the terms
metadata['totalWords'] = 0 #totally number of words in the collection
metadata['termINall'] = {} #mapping from term name to collection frequency

#look up table for reading file
#key is term, and value is where in the file to find the term
lookup_table = {}
loopup_comp = {}


### create the inverted list, update the metadata###

################################################################################

#get a new instance of the list object 
#key: term name value: a list of positions for each document
invertls = {}
#get the list of documents 
docs = data['corpus']
#to go through each doc
for x in range (0, len(docs)):
    curr_doc = docs[x]
    metadata['collectionLen'] += 1
    
    #remember the whole scnen and play name for later use
    metadata['docid'].append(curr_doc['sceneId'])
    #the play name
    playId = curr_doc['playId']
    #the whole play-scene name 
    sceneId = curr_doc['sceneId']   
    #use this as the key/ index in the list
    sceneNum = curr_doc['sceneNum']
    #split into words
    text = curr_doc['text'].split()
    metadata['docLen'].append([len(text), sceneId])
    metadata['totalWords'] += len(text)
    #loop through each word
    for i in range(0, len(text)):
        
        #get the word
        word = text[i]
        #if the term has been created
        if(invertls.get(word)!= None):
            #add the word to the term
            #object to list to term #given the term name, the position and in which doc we find it
            invertls[word].addWord(text[i], i, x)
            #metadata['termINall'][text[i]] += 1
            
        #else create the term and add
        else:
            #when this is a new word
            metadata['terms'].append(text[i])
            #create a term
            newterm = termList(text[i])
            newterm.addWord(text[i], i, x)
            #metadata['termINall'][text[i]] += 1
            invertls[text[i]] = newterm
    
currentplay = docs[0]['playId']
metadata['playLen'].append([0, currentplay])
currentplace = 0
#list of pairs[play name, play name]
#loop throught the docs to get length of each play
for doc in docs:
    #get the play this doc is in
    docplay = doc['playId']
    #get the index
    docnum = doc['sceneNum']
    #if this is a new play
    if(currentplay != docplay):
        currentplace += 1
        metadata['playLen'].append([len(doc['text'].split()), docplay])
        currentplay = docplay
        
        
    else:
        #add the doc length to the play len
        metadata['playLen'][currentplace][0] += len(doc['text'].split())
        



In [158]:
### find the avg scene len, long/short play/scene ###

################################################################################

#play length
metadata['playLen'].sort()
print('Shortest play: ', metadata['playLen'][0])

print('longest play: ', metadata['playLen'][-1])

#scene length
metadata['docLen'].sort()
print('Shortest scene: ', metadata['docLen'][0])
print('longest scene: ', metadata['docLen'][-1])

#ave scene
print('Ave scene length: ', metadata['totalWords']/metadata['collectionLen'])
#print(metadata['totalWords'])

Shortest play:  [16415, 'comedy_of_errors']
longest play:  [32867, 'hamlet']
Shortest scene:  [47, 'antony_and_cleopatra:2.8']
longest scene:  [7988, 'loves_labors_lost:4.1']
Ave scene length:  1199.5561497326203


In [116]:
### vbyte compress and delta compress ###

################################################################################
#decimal - binary - add one(last byte)/zero before every 7 bit - 16 

        
def vbyte_encode(data):
    ls = []
    
    for x in range (0,len(data)):
        value = data[x]
        ls.append(0)
        if(value <= 128):
            ls[x] = value +128
            
        else:
            count = 0
            while(value >= 128):
                temp = value
                value = value>>7
            
                rest = temp - (value<<7)
                #print(bin(rest))
                if(count == 0):
                    #do notadd
                    count+=1
                    ls[x] += rest+128
                    #print(bin((rest<<(8*count))))
                    #ls[x] += (rest<<(8*count))
                
                else:
                    #add the value with zero before it
                    #print('else  ', (rest<<(8*count)))
                    ls[x] += (rest<<(8*count))
            
            #add 128 to the last 7 bits
            ls[x] += (value<<(8*count))
        
    return ls

def vbyte_decode(byte_input):
    intls = []
    for x in range(0, len(byte_input)):
        intls.append(0)
        #myint = int(byte_input[x], 2)
        myint = byte_input[x]
        
        #while we do not reach the last eight digits where the first is a one
        count = 0
        shift = 0
        #print(rest)
        while (myint >= 128):
            temp = myint
            myint = myint>>(7+shift)
            #print('myintfirst: ', myint)
            rest = temp - (myint<<(7+shift))
            shift = 1
            
            if(count == 0):
                #print(bin(rest))
                intls[x] += rest
                count += 1
                
            else:
                #add the rest 
                intls[x] += (rest<<(7*count))
                count += 1
            
        myint = myint>>1
        #print(myint)
        intls[x] += (myint<<(7*count))
        
    return intls
    
    
    
    
    
def delta_encode(ints):
    encoded = []
    encoded.append(ints[0])
    for x in range (1, len(ints)):
        encoded.append(ints[x] - ints[x-1])
        
    return encoded
    
    
    
def delta_decode(ints):
    decoded = []
    decoded.append(ints[0])
    for x in range (1, len(ints)):
        decoded.append(decoded[x-1] + ints[x]) 
        
    return decoded
     

#do the two encoding above given a term list
def compressTerm(term):
    for key in term:
        #compresss each pos list 
        term[key] = vbyte_encode(delta_encode(term[key]))
        #print(term[key])
        
    return term 


#given the bytes array
def decompress(mybytes):
    
    return delta_decode(vbyte_decode(mybytes))
    
#a = compressTerm({1:[2,4,5]})
#decompress(a)
    

In [161]:
### WRITE to disk and update the look up table ###

################################################################################

filename = 'invli.bin'
comp_filename = 'comp_invli.bin'
filepos = 0
with open(filename, 'w') as f:
    #for each of the term
    for key in invertls:
        f.write(str(invertls[key]))
        #print(str(invertls[key]))
        lookup_table[key] = [f.tell(), f.tell()-filepos]
        filepos = f.tell()
         
        
        
filepos_c = 0
with open(comp_filename, 'w') as f:
    #for each of the term
    for key in invertls: 
        #original term's doc-position dictionary
        term_original = invertls[key]
        #compressed version
        comp = compressTerm(term_original)
        #print(comp)
        f.write(str(comp))
        lookup_table[key] = [f.tell(), f.tell()-filepos_c]
        filepos_c = f.tell()
        
        

        
#read from the disk by term name
#return the inverted list for that term 
def readTerm(termname, filename):
    #get where and what to read
    start, size = lookup_table[termname]
    with open(filename, 'r') as f:
        f.seek(start)
        term_jstr = f.read(size)
        
    return term_jstr


{0: [0], 1: [0], 2: [0], 3: [0], 4: [0], 5: [0], 6: [0], 7: [0], 8: [0], 9: [0], 10: [0], 11: [0], 12: [0], 13: [0], 14: [0], 15: [0, 1399], 16: [0], 17: [0], 18: [0], 19: [0], 20: [0], 21: [0], 22: [0], 23: [0], 24: [0], 25: [0], 26: [0], 27: [0], 28: [0, 1696], 29: [0], 30: [0], 31: [0], 32: [0], 33: [0], 34: [0], 35: [0], 36: [0], 37: [0], 38: [0], 39: [0], 40: [0], 41: [0], 42: [0], 43: [0, 1446], 44: [0], 45: [0], 46: [0], 47: [0], 48: [0], 49: [0, 708], 50: [0], 51: [0], 52: [0], 53: [0], 54: [0], 55: [0], 56: [0], 57: [0], 58: [0], 59: [0], 60: [0], 61: [0], 62: [0], 63: [0], 64: [0], 65: [0], 66: [0], 67: [0], 68: [0], 69: [0], 70: [0], 71: [0], 72: [0], 73: [0], 74: [0], 75: [0], 76: [0], 77: [0], 78: [0], 79: [0], 80: [0], 81: [0], 82: [0], 83: [0], 84: [0], 85: [0], 86: [0], 87: [0], 88: [0], 89: [0], 90: [0], 91: [0], 92: [0], 93: [0], 94: [0, 150], 95: [0], 96: [0], 97: [0], 98: [0], 99: [0], 100: [0], 101: [0], 102: [0], 103: [0], 104: [0], 105: [0, 347], 106: [0], 107: [

{0: [29, 79, 96, 177, 180, 191, 195, 216, 226, 266, 289, 310, 327, 474, 512, 553, 578, 641, 683, 691, 753, 757, 761, 800, 922, 986, 991, 1053, 1072, 1117, 1182, 1185, 1194, 1251, 1257, 1282, 1348, 1350, 1404, 1459, 1481, 1530], 1: [14, 30, 59, 84, 109, 125, 140, 166, 225, 241, 254, 301, 304, 309, 312, 319, 322, 338, 343, 362, 381, 388, 408, 436, 534, 562, 634, 654, 670, 721, 755, 807, 824, 838, 855, 863, 874, 883], 2: [5, 73, 110, 113, 135, 146, 149, 162, 198, 203, 216, 241, 259, 308, 318, 422, 443, 493, 531, 605, 615, 639, 651, 722, 746, 774, 937, 1022, 1025, 1045, 1047, 1080, 1098, 1144, 1153, 1171, 1199, 1207, 1248, 1252, 1258, 1278, 1455, 1473, 1480, 1511, 1538, 1550, 1603, 1643, 1661, 1761, 1767, 1784, 1805, 1813, 1826, 1844, 1851, 1863, 1866, 1901, 1907, 1915, 1922, 1933, 1936, 1975, 1989, 1997, 2029, 2138, 2154, 2162, 2208, 2210, 2253, 2263, 2365, 2392, 2555, 2579, 2659, 2696, 2754, 2800, 2812, 2828, 2882, 2888, 2935, 2962, 3004, 3039, 3072, 3098, 3119, 3151, 3177, 3215, 3243, 3

{0: [88, 260], 1: [308], 3: [238, 529, 1904], 5: [116, 230, 673, 801, 828, 839, 1078, 1163, 1193, 1244, 1249, 1279, 1318, 2106, 2535, 2738], 6: [371], 8: [378, 4185], 9: [16], 10: [899, 1871], 13: [535], 33: [777], 115: [140, 143], 156: [79, 225], 166: [160, 174], 253: [19, 27, 64, 91], 261: [13, 73], 335: [26, 76], 351: [14, 23, 118, 241], 390: [1560], 541: [172], 547: [712], 550: [21], 576: [150, 157], 694: [9, 19, 351], 703: [105]}
{0: [89], 5: [2059, 2065], 8: [115, 2184, 2294], 9: [1501], 11: [1415], 13: [952], 14: [2973], 16: [142], 18: [300, 303, 306, 412, 480], 21: [61, 434, 439, 1162, 2089, 2191], 23: [2149, 2208, 2494], 24: [172, 1205], 25: [896], 27: [636], 28: [1796], 29: [446], 31: [574], 36: [798], 37: [276], 38: [1213, 2547, 2568], 39: [669, 866], 42: [64], 43: [401, 480, 548, 559], 44: [390, 1153], 45: [291], 46: [479, 1460, 1829], 47: [773], 48: [36, 284], 49: [764], 52: [450], 57: [899, 1346], 58: [643], 59: [396], 60: [946], 61: [114, 2765], 64: [12, 174, 640], 70: [

{0: [236], 5: [1367], 11: [731], 19: [634], 25: [1025], 38: [462], 46: [777, 869, 876, 1165], 62: [536], 70: [367], 77: [803], 84: [708], 89: [503], 102: [2770], 103: [39], 112: [221], 114: [825, 872, 980], 118: [3720], 133: [312, 317], 140: [1606], 143: [18, 48, 226, 257], 163: [92], 180: [245], 186: [87], 188: [1497], 195: [598], 221: [30], 265: [188], 392: [1601], 406: [703], 415: [641], 479: [504], 587: [394, 415], 590: [2269], 602: [321, 1317, 1468], 605: [103, 179], 606: [163], 607: [132, 285, 358], 610: [128, 131], 612: [409, 537, 591, 599, 609], 614: [156], 616: [180, 190, 206, 313], 617: [2544, 2564, 2675, 2750, 3319], 624: [629], 625: [905, 1282], 628: [882], 629: [2231], 631: [1519], 634: [1671], 635: [326], 663: [293], 693: [1944], 714: [3176], 725: [298]}
{0: [238], 143: [305], 415: [633], 620: [187], 664: [254]}
{0: [239, 258, 855, 1229, 1326], 1: [269], 2: [256, 438, 912, 1872, 1969, 2371, 2377, 2537, 2577, 3025, 3045, 3049, 3473], 3: [1914, 1978, 2491], 5: [108, 785, 79

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

{398: [455], 491: [895]}
{398: [1378]}
{398: [1381]}
{398: [1971], 570: [3059], 731: [432]}
{398: [2006]}
{398: [2031]}
{398: [2504]}
{398: [2568]}
{399: [38], 479: [470], 538: [504]}
{399: [275]}
{399: [374]}
{399: [619]}
{400: [156]}
{400: [670], 672: [430]}
{400: [803]}
{400: [826]}
{400: [828], 484: [113]}
{400: [830], 573: [1232], 616: [765], 633: [173], 734: [1966]}
{400: [838], 430: [1140], 683: [1102], 732: [1246, 1248]}
{400: [1469]}
{400: [1484]}
{400: [1620], 615: [627]}
{400: [1644]}
{400: [1837], 475: [381], 725: [637]}
{400: [1898]}
{400: [2154, 2604], 421: [3044]}
{400: [2267]}
{400: [2342], 432: [475], 494: [225], 515: [781]}
{400: [2787]}
{401: [100], 417: [2411]}
{401: [210], 580: [179], 590: [1015], 661: [807]}
{401: [217]}
{401: [223]}
{401: [228]}
{401: [694]}
{402: [145], 459: [94], 705: [2643], 707: [427], 717: [716]}
{402: [177]}
{403: [252]}
{403: [385]}
{403: [927]}
{403: [942], 645: [443], 744: [319]}
{403: [945]}
{403: [1182], 600: [459]}
{404: [3, 8, 16, 14

{456: [852]}
{456: [1046], 474: [3530], 491: [573], 570: [246]}
{457: [93]}
{457: [106], 459: [32, 557], 463: [924], 474: [747]}
{457: [124]}
{457: [249], 603: [192], 676: [1386]}
{457: [631]}
{457: [646], 535: [1671], 570: [2071]}
{457: [674]}
{457: [744]}
{457: [761]}
{457: [800]}
{457: [932], 463: [687]}
{457: [1239], 491: [4556]}
{458: [26]}
{458: [100]}
{458: [102], 474: [1123]}
{458: [122]}
{458: [149], 572: [4098], 639: [228], 642: [2822], 657: [1828]}
{459: [179], 483: [1064]}
{459: [185]}
{459: [204], 571: [297], 572: [1732]}
{459: [286], 467: [952, 1183], 474: [4288]}
{459: [486, 521]}
{459: [642], 465: [33]}
{459: [644], 465: [35]}
{460: [184], 482: [1030]}
{460: [237], 682: [827]}
{460: [286]}
{460: [287]}
{460: [385], 474: [170]}
{460: [741]}
{461: [13, 18, 106, 411, 599, 960], 464: [7, 263, 267, 377, 405, 549, 556, 593, 738, 765, 827], 466: [7], 467: [6, 11, 16, 190, 251, 404, 429, 449, 579, 606, 670, 965, 968, 982, 1047, 1128, 1186, 1245, 1392, 1465, 1530, 1602, 1750, 17

{532: [436], 534: [10, 150, 170, 285, 337, 374, 450], 539: [8, 795, 798, 822, 834, 861, 888, 906, 919, 944, 959, 992, 1019, 1040, 1048, 1074, 1129, 1204, 1235], 540: [175, 338, 348, 356, 377, 495, 518, 539, 551, 590, 604, 621, 646, 685, 785, 837, 838, 855, 864, 880, 904, 1238, 1239, 1245, 1289], 542: [58]}
{532: [456]}
{532: [463]}
{532: [510]}
{532: [515]}
{532: [517]}
{533: [11, 62, 63, 105, 111], 538: [31, 50, 72, 1104], 566: [11, 67, 89], 567: [206, 219, 249, 317, 335, 349]}
{533: [91]}
{534: [27]}
{534: [172], 652: [167, 193, 215, 281], 660: [8, 11, 245, 745, 754, 755, 768, 769, 805, 815, 876, 940, 1112, 1221, 1222, 1256], 663: [53, 54, 184, 268, 322, 370, 509, 613], 665: [669, 745, 778, 811, 886, 952, 1012, 1038, 1062], 667: [51, 167, 225, 1610]}
{534: [224]}
{535: [125]}
{535: [136, 158], 536: [286, 374, 377], 541: [7, 29, 68, 110, 241, 259, 305], 731: [838, 890], 732: [29, 39, 104, 146], 734: [1863, 1885], 737: [37, 80]}
{535: [145, 165, 904, 936, 1542, 1555, 1563, 1601, 1641, 

{609: [497]}
{609: [572]}
{609: [617]}
{609: [663], 671: [898]}
{610: [72], 661: [320]}
{611: [393]}
{611: [476]}
{611: [651]}
{612: [22]}
{612: [38], 726: [111], 728: [123]}
{612: [728]}
{612: [931]}
{612: [1012]}
{612: [1018]}
{612: [1023]}
{612: [1097, 1103]}
{613: [209]}
{613: [511]}
{613: [765]}
{613: [838]}
{613: [864]}
{613: [902], 743: [1196]}
{613: [1057]}
{613: [1298]}
{613: [1303], 668: [106]}
{613: [1797, 1802, 1810, 1873, 1890, 1927, 1960, 1992, 2204, 2238], 617: [15, 128, 922]}
{613: [2077]}
{613: [2122]}
{613: [2222]}
{613: [2313], 617: [11]}
{613: [2353]}
{613: [2421]}
{614: [157]}
{614: [176]}
{614: [278]}
{615: [408], 617: [906, 1030, 1036, 1237, 1290, 1441, 1470], 619: [2357]}
{615: [429]}
{615: [433]}
{616: [152]}
{616: [158]}
{616: [227]}
{616: [337]}
{616: [596]}
{616: [760]}
{617: [234]}
{617: [265]}
{617: [663]}
{617: [1116]}
{617: [1253]}
{617: [1605]}
{617: [1621]}
{617: [1917, 1923, 2869, 2871, 2928, 2934]}
{617: [2556]}
{617: [2716], 722: [1749]}
{617: [3416

{684: [755], 699: [244]}
{685: [83]}
{685: [292]}
{686: [63]}
{686: [67]}
{686: [69], 688: [119, 120, 129, 198]}
{686: [71]}
{686: [82]}
{686: [87]}
{686: [89]}
{686: [90]}
{686: [93]}
{686: [95]}
{686: [104]}
{686: [155]}
{686: [175]}
{686: [187]}
{686: [199]}
{686: [201], 743: [2167]}
{686: [259]}
{686: [301]}
{686: [312]}
{687: [278]}
{688: [45]}
{689: [178]}
{690: [51]}
{691: [128]}
{691: [263]}
{691: [508]}
{692: [56], 714: [1853]}
{692: [152]}
{692: [160]}
{692: [273]}
{692: [295]}
{692: [416]}
{692: [434]}
{692: [464]}
{692: [540]}
{692: [696]}
{693: [126, 348, 351, 382, 436, 439, 462, 478, 751]}
{693: [168]}
{693: [336, 364, 491]}
{693: [338]}
{693: [339]}
{693: [340]}
{693: [342]}
{693: [390, 446, 526]}
{693: [393, 444]}
{693: [401, 517]}
{693: [434]}
{693: [455]}
{693: [541]}
{693: [549]}
{693: [555]}
{693: [566, 649, 718]}
{693: [577, 683]}
{693: [617]}
{693: [675]}
{693: [773]}
{693: [854]}
{693: [1085]}
{693: [1159]}
{693: [1255], 714: [3174], 729: [7], 741: [145]}
{693: [

In [88]:
### dice's coefficient ###

################################################################################

#get the best dice score term for the input term
def best_dice(term):
    #if our term only appear at the end of the test
    
    best_score = -100
    best_term = None
    #for each of the terms in the list
    for key in invertls:
        term_compare = invertls[key]
        d_ef = getD_coef(term, term_compare)
        #when we find a higher score
        if(d_ef>best_score):
            best_score = d_ef
            best_term = term_compare
        
    #print(best_score)
    return best_term
            
        


#get the coef for phrase12 and phrase21
def getD_coef(term1, term2):
    n1 = term1.collection_f
    n2 = term2.collection_f
    n12 = conFrequency(term1, term2)
    
    
    
    return n12 / (n1+n2)

#get the phrase frequency, input terms have the same current doc
def phraseInDoc(term1, term2):
    cur_docID = term1.currentDoc
    t1_pos = getDocPos(term1)
    t2_pos = getDocPos(term2)
    
    count = 0
    for x in range (0, len(t1_pos)):
        
        if(t1_pos[x]+1 in t2_pos or t1_pos[x]-1 in t2_pos):
            count += 1
            
    return count
        
    
    

#takes two term objects and find frequency they show up together
#we only care about when term1 comes in the front
def conFrequency(term1, term2):
    #get the two doc-position dictionaries
    t1_docs = term1
    t2_docs = term2
    
    #get the docid list for terms
    t1_docid1 = list(t1_docs.keys())
    t2_docid2 = list(t2_docs.keys())
    
    #find the intersection of the two
    common_doc = list(set(t1_docid1).intersection(t2_docid2))

    count = 0
    #print(common_doc)
    #in each doc where two terms both appear
    for docnum in common_doc:
        t1_pos = t1_docs[docnum]
        t2_pos = t2_docs[docnum]
        
        #increment positions by one
        t1_posIN = np.array(t1_pos) + 1
        t2_posIN = np.array(t2_pos) + 1
        
        #add t1t2 and t2t1
        #len(list(set(t1_pos).intersection(t2_posIN))) + 
        count += len(list(set(t2_pos).intersection(t1_posIN)))
        
    return count

        
    
testterm = best_dice(invertls['little'])    


In [89]:
print(testterm.name)

while


In [138]:
### randomly select 7 words from the vocabulary read from file and search ###

################################################################################

#given a list of words, return the docID with the highest score
def searchwords(words, meta, filename):
    #a list to remember the best 10 docs
    good_docs = []
    
    doclist = meta['docid']
    #go through each document 
    for x in range (0, len(doclist)):
        score = 0
        for oneword in words:
            word_invls = readTerm(oneword, filename)
            pos_word = word_invls[x]
            #get the score
            #add to the total score
            score += len(pos_word)
            
        #add the score and the doc
        good_docs.append([score, doclist[x]])
        #sort the list
        good_docs.sort()
        #if the qsize is 10, remove the least element    
        if(good_docs.qsize() == 11):
            good_docs.pop(0)
            
    return good_docs
            
        
#return a list of term names        
def getRanTerm(n, meta):
    #get the list of term names from metadata
    termNs = meta['terms']
    #randomly select 7 of the names
    sele_termN = random.sample(termNs, n)
    
    return sele_termN
    
    

#get 7*100 rand term names and store in file
#Query les:
#these should contain the 100 sets of 7 terms, one set per line, in a plain text le
#and the 100 sets of 7
#two term phrases, one set per line
with open('single_wordQ.txt', 'a') as f:
    
    for x in range(0, 100):
        st= ''
        terms7 = getRanTerm(7, metadata)
        #for each of the 7 terms
        for term in terms7:
            st = str(term)+' '+st 
        f.write(st + '\n' )


print('Done')


Done


In [ ]:
def read7():
    a = []
    with open('single_wordQ.txt', 'r') as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    for line in content:
        line = line.split()
        
        a.append(line)
    return a

singlewords = read7()
with open('two_wordQ.txt', 'a') as f:
    for x in range(0, 100):
        st= ''
        terms7 = singlewords[x]
        #for each of the 7 terms
        for term in terms7:
            st = str(term)+' '+ best_dice(invertls[term]).name + ' '+st 
            
        print(x)
        f.write(st + '\n' )


print('Done')


In [148]:
def read14():
    a = []
    with open('two_wordQ.txt', 'r') as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    for line in content:
        line = line.split()
        
        a.append(line)
    return a


twowords = read14()
lstime_comp = [] #[[7,14], [7,14]......]
lstime_uncomp = []
docls7c = []
docls14c = []
docls7uc = []
docls14uc = []
if(print(sys.argv)[1] == 'comp'):
    #run the compressed search on 7 and 14
    for x in range (0, 100):
        starttime7 = time.time()
        #get the largest score
        doc7 = searchwords(singlewords[x], metadata, comp_filename)[-1]
        endtime7 = time.time()
        docls7c.append(doc7)
        
        starttime14 = time.time()
        #get the largest score
        doc14 = searchwords(twowords[x], metadata, comp_filename)[-1]
        endtime14 = time.time()
        docls14c.append(doc14)
        #time difference
        lstime_comp.append([endtime7-starttime7, endtime14-starttime14])
#when not compressed
else:
    #run the compressed search on 7 and 14
    for x in range (0, 100):
        starttime7u = time.time()
        #get the largest score
        doc7u = searchwords(singlewords[x], metadata, filename)[-1]
        endtime7u = time.time()
        docls7uc.append(doc7u)
        
        starttime14u = time.time()
        #get the largest score
        doc14u = searchwords(twowords[x], metadata, filename)[-1]
        endtime14u = time.time()
        docls14uc.append(doc14u)
        #time difference
        lstime.append([endtime7u-starttime7u, endtime14u-starttime14u])


100